<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#오차역전파-적용한-신경망-구현" data-toc-modified-id="오차역전파-적용한-신경망-구현-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>오차역전파 적용한 신경망 구현</a></span></li><li><span><a href="#수치미분-활용해-오차역전파법으로-구한-기울기-값-검증" data-toc-modified-id="수치미분-활용해-오차역전파법으로-구한-기울기-값-검증-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>수치미분 활용해 오차역전파법으로 구한 기울기 값 검증</a></span></li><li><span><a href="#오차역전파를-사용한-MNIST-데이터-학습-구현" data-toc-modified-id="오차역전파를-사용한-MNIST-데이터-학습-구현-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>오차역전파를 사용한 MNIST 데이터 학습 구현</a></span></li></ul></div>

In [1]:
import numpy as np

class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        return x * y
    
    def backward(self, d_out):
        # 순전파 시 입력을 서로 바꾸어서 곱해줌!
        dx = d_out * self.y
        dy = d_out * self.x
        return dx, dy
    
apple_box = 100
apple_box_num = 2
tax = 1.1

# 곱셈계층
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파 수행
apple_box_price = mul_apple_layer.forward(apple_box, apple_box_num)
price = mul_tax_layer.forward(apple_box_price, tax)
print('순전파 수행 후 지불해야 할 최종 금액:', price)
print()

# 역전파 수행(순전파와 반대 순서로 호출)
d_price = 1
d_apple_box_price, d_tax = mul_tax_layer.backward(d_price)
d_apple_box, d_apple_box_num = mul_apple_layer.backward(d_apple_box_price)
print('역전파 수행 후 각 변수의 변화량 값')
print('사과 박스 가격:', d_apple_box)
print('사과 박스 개수:', d_apple_box_num)
print('소비자세:', d_tax)

순전파 수행 후 지불해야 할 최종 금액: 220.00000000000003

역전파 수행 후 각 변수의 변화량 값
사과 박스 가격: 2.2
사과 박스 개수: 110.00000000000001
소비자세: 200


In [2]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        return x + y
    
    def backward(self, d_out):
        dx = d_out * 1
        dy = d_out * 1
        return dx, dy

- 블로깅 완료
---

In [3]:
import numpy as np

In [4]:
import numpy as np

# Relu 활성함수의 역전파 계층 만들기
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x: np.array):
        self.mask = (x <= 0) # 배열 원소값이 0이하인 값들에 대한 Boolean 인덱싱
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self, d_out):
        d_out[self.mask] = 0 # 순전파 시 입력값이 0이하인 데이터에 대한 국소적인 미분값은 0으로 전달하도록 변환
        dx = d_out
        
        return dx

In [5]:
# Sigmoid 활성함수의 역전파 계층 만들기 -> 결국 최종 변화량은 1(d_out) * y(1-y)가 됨!
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x: np.array):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        
        return out
    
    def backward(self, d_out):
        dx = d_out * self.out(1 - self.out)
        
        return dx

In [6]:
a = np.array([1,2,3])
np.expand_dims(a, axis=1)

array([[1],
       [2],
       [3]])

In [7]:
# Affine(행렬 곱) 계층 구현 -> 4차원인 경우 고려한 코드랑 차이가 있음!
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        # out = x * W + b
        self.x = x
        out = np.matmul(x, self.W) + self.b
        return out
    
    def backward(self, d_out):
        dx = np.matmul(d_out, self.W.T)
        self.dW = np.matmul(self.x.T, d_out)
        self.db = np.sum(d_out, axis=0)
        
        return dx

In [8]:
# Sofmtax-with-Loss 계층 구현
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None   # prediction
        self.t = None   # label
        
    def softmax(x):
        max_x = np.max(x)
        exp_x = np.exp(x - max_x)
        exp_x_sum = np.sum(exp_x)
        
        return exp_x / exp_x_sum
    
    def cross_entropy_error(y, t):
        if y.ndim == 1:
            t = t.reshape(1, t.size)
            y = y.reshape(1, y.size)

        # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
        if t.size == y.size:
            t = t.argmax(axis=1)

        batch_size = y.shape[0]
        return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

    def forward(self, x, t):
        self.t = t
        self.y = self.softmax(x)
        self.loss = self.cross_entropy_error(self.y, self.t)
        
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx

---
## 오차역전파 적용한 신경망 구현

In [48]:
import numpy as np

#===============
# 필요한 함수들 구현
#===============
def sigmoid(x: np.array):
    return 1 / (1 + np.exp(-x))

def relu(x: np.array):
    return np.maximum(0, x)

def softmax(x: np.array):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T
    x = x - np.max(x)
    return np.exp(x) / np.sum(np.exp(x))

def cross_entropy_error(y: np.array, t: np.array):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, t.size)
    # 학습 데이터의 레이블이 One-hot 형태라면 Label 형태로 바꾸어주기
    if t.size == y.size:
        t = t.argmax(axis=1)
    
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

# 수치 미분 함수
def numerical_gradient(f, x):
    h = 1e-4
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        # f(x+h)
        x[idx] = float(tmp_val) + h
        fx1 = f(x)
        # f(x-h)
        x[idx] = float(tmp_val) - h
        fx2 = f(x)
        
        grad[idx] = (fx1 - fx2) / (2*h)
        
        x[idx] = tmp_val
        it.iternext()
        
    return grad

#===============
# 필요한 계층들 구현 
#===============
# 1.Relu
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x: np.array):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx
    
# 2.Sigmoid
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self, x: np.array):
        out = sigmoid(x)
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * self.out * (1 - self.out)
        return dx
    
# 3.Affine
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        
        self.x = None
        self.original_x_shape = None
        self.dW = None
        self.db = None
        
    def forward(self, x: np.array):
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x
        
        out = np.matmul(self.x, self.W) + self.b
        return out
    
    def backward(self, dout):
        dx = np.matmul(dout, self.W.T)
        self.dW = np.matmul(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        dx = dx.reshape(*self.original_x_shape)
        return dx

# 4.Softmax-with-Loss
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
        
    def forward(self, x: np.array, t: np.array):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss  # 순전파 시, Loss 계층이 가장 마지막이니까 loss를 리턴
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx

In [51]:
import numpy as np
from collections import OrderedDict

#===========================================================
# -> dx는 입력 데이터의 변화량을 의미하는데, 입력 데이터는 변경시킬 수 없음! 
# -> 우리는 W,b만을 변경시켜야 하므로 마지막 gradient 메소드에서 반환되는 dout 변수는 파라미터 변화량 계산하기 위한 정의용인듯!
#===========================================================

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        #================================================================================
        #계층 생성 -> 순서가 있는 OrderedDict 객체 사용(Why?-> 순전파/역전파 호출 계층 순서가 반대기 떄문!)
        #================================================================================
        self.layers = OrderedDict()
        # 1.Affine & Activtion(Relu)
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        # 2.Affine & Activation(Softmax)
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.lastlayer = SoftmaxWithLoss()
        
    # 순전파 시 예측
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
            
    # 순전파 시 Softmax & 손실함수
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastlayer.forward(y, t) # 이떄, y는 layer거쳐 나온 출력값임!
    
    # 정확도 측정
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        # 정답도 One-hot 형태라면 레이블 형태로 변경
        if t.ndim != 1: t = np.argmax(t, axis=1) 
        
        accuracy = np.sum(y == t) / float(y.shape[0])
        return accuracy
    
    # 역전파
    def gradient(self, x, t):
        # 순전파 수행
        self.loss(x, t)
        
        # 역전파 수행
        dout = 1
        dout = self.lastlayer.backward(dout)  # Softmax-with-Loss 계층 역전파
        
        layers = list(self.layers.values())
        layers.reverse() # 레이어 순서 뒤집기
        for layer in layers:
            dout = layer.backward(dout)
        
        # 역전파 수행한 후의 파라미터 변화량 저장
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        
        return grads
    
    # 수치미분
    def numerical_gradient(self, x, t):
        loss_W = lambda w: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads

## 수치미분 활용해 오차역전파법으로 구한 기울기 값 검증

In [55]:
from dataset.mnist import load_mnist

# load data
(X_train, y_train), (X_test, y_test) = load_mnist(normalize=True, one_hot_label=True)

# model
network = TwoLayerNet(input_size=28*28, hidden_size=50, output_size=10)

# Batch
X_batch = X_train[:3]
y_batch = y_train[:3]

# 수치미분
grad_numerical = network.numerical_gradient(X_batch, y_batch)
grad_propagation = network.gradient(X_batch, y_batch)

for key in grad_numerical.keys():
    diff = np.mean(np.abs(grad_numerical[key] - grad_propagation[key]))
    print('key:', key, 'diff:', diff)

key: W1 diff: 3.7018114837401005e-10
key: b1 diff: 2.196205218836381e-09
key: W2 diff: 4.65686156334598e-09
key: b2 diff: 1.4011442090039461e-07


## 오차역전파를 사용한 MNIST 데이터 학습 구현

In [65]:
import numpy as np
from dataset.mnist import load_mnist

(X_train, y_train), (X_test, y_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=28*28, hidden_size=50, output_size=10)

steps = 10000
train_size = X_train.shape[0]
batch_size = 100
learning_rate = 0.1


train_loss = []
train_acc = []
test_acc = []

# Mini-batch로 학습
for i in range(steps):
    batch_mask = np.random.choice(train_size, batch_size)
    X_batch = X_train[batch_mask]
    y_batch = y_train[batch_mask]
    
    # 오차역전파로 학습하면서 파라미터 변화량 get!
    grad = network.gradient(X_batch, y_batch)
    
    # 얻은 변화량으로 SGD(경사 하강) 수행!
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    # SGD 수행 후 다시 학습 데이터에 예측해서 Loss 계산
    loss = network.loss(X_batch, y_batch)
    train_loss.append(loss)
    
    if i % 1000 == 0:
        tr_acc = network.accuracy(X_train[batch_mask], y_train[batch_mask])
        te_acc = network.accuracy(X_test, y_test)
        train_acc.append(tr_acc)
        test_acc.append(te_acc)
        print(f'{i+1}번째 학습 후 Train Acc:', round(tr_acc, 3))
        print(f'{i+1}번째 학습 후 Test Acc:', round(te_acc, 3))
        print()

1번째 학습 후 Train Acc: 0.15
1번째 학습 후 Test Acc: 0.098

1001번째 학습 후 Train Acc: 0.95
1001번째 학습 후 Test Acc: 0.922

2001번째 학습 후 Train Acc: 0.97
2001번째 학습 후 Test Acc: 0.94

3001번째 학습 후 Train Acc: 0.99
3001번째 학습 후 Test Acc: 0.95

4001번째 학습 후 Train Acc: 0.97
4001번째 학습 후 Test Acc: 0.957

5001번째 학습 후 Train Acc: 0.95
5001번째 학습 후 Test Acc: 0.961

6001번째 학습 후 Train Acc: 0.98
6001번째 학습 후 Test Acc: 0.966

7001번째 학습 후 Train Acc: 1.0
7001번째 학습 후 Test Acc: 0.966

8001번째 학습 후 Train Acc: 0.99
8001번째 학습 후 Test Acc: 0.967

9001번째 학습 후 Train Acc: 0.99
9001번째 학습 후 Test Acc: 0.97

